In [1]:
import pandas as pd
import numpy as np 

df_after_member = pd.read_csv(r"data\newest\subset_afterT_member.csv")
df_after_nonmember = pd.read_csv(r"data\newest\subset_afterT_nonmember.csv")

C:\Users\manue\AppData\Local\Temp\ipykernel_23100\2497486820.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [6]:
len(df_after_nonmember)

5457450

In [5]:
df_after_member.iloc[df_after_member['Quantity'].idxmin()]

Quantity                 -300.0
Rev                     -314.29
HL3_101                       0
HL3_102                       0
HL3_103                       0
HL3_104                       0
HL3_105                       1
HL3_106                       0
HL3_107                       0
HL3_108                       0
HL3_109                       0
HL3_110                       0
HL3_111                       0
HL3_112                       0
HL3_114                       0
HL3_115                       0
HL3_116                       0
HL3_117                       0
HL3_118                       0
HL3_119                       0
HL3_120                       0
HL3_121                       0
HL3_122                       0
HL3_123                       0
HL3_124                       0
HL3_125                       0
HL3_126                       0
HL3_127                       0
HL3_128                       0
HL3_129                       0
HL3_130                       0
HL3_131 

In [3]:
# Preparing data

import xgboost as xgb
from sklearn.model_selection import train_test_split

# Used to train ML models
data = df_after_nonmember.drop(['Rev', 'Date', 'T'], axis=1)
data['Operation Type'] = data['Operation Type'] - 1         # changing from (1, 2) to binary
label = df_after_nonmember.Rev

del df_after_member
del df_after_nonmember

## XGBoost

In [4]:
# Split
from sklearn.model_selection import train_test_split
X_temp, X_test, y_temp, y_test = train_test_split(data, label, test_size = 0.1, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size = 0.1, random_state=42)

X_test.to_csv(r'data\newest\nonmembers_test_X.csv')
y_test.to_csv(r'data\newest\nonmembers_test_y.csv')

In [5]:
# Data types needed for XGB
dtrain = xgb.DMatrix(X_train, label=y_train)
dval = xgb.DMatrix(X_val, label=y_val)

evallist = [(dtrain, 'train'), (dval, 'eval')]

del X_temp
del y_temp
del X_train
del y_train
del X_test
del y_test

In [13]:
# Training
# TODO: hyper opt
param = {'max_depth': 5, 'eta': 0.5, 'objective': 'reg:squarederror', 'colsample_bytree':0.8}

num_round = 400
bst = xgb.train(param, dtrain, num_round, evals=evallist, early_stopping_rounds=30)
bst.save_model(r'models\newest\prognostic_model_xgb.json')

[0]	train-rmse:24.61448	eval-rmse:24.10271
[1]	train-rmse:18.96197	eval-rmse:18.54330
[2]	train-rmse:14.33956	eval-rmse:13.89106
[3]	train-rmse:12.79752	eval-rmse:12.40385
[4]	train-rmse:12.11716	eval-rmse:11.72977
[5]	train-rmse:11.56241	eval-rmse:11.19513
[6]	train-rmse:11.40655	eval-rmse:11.04754
[7]	train-rmse:11.24410	eval-rmse:10.88120
[8]	train-rmse:11.06721	eval-rmse:10.69000
[9]	train-rmse:10.98310	eval-rmse:10.60407
[10]	train-rmse:10.93486	eval-rmse:10.55864
[11]	train-rmse:10.67749	eval-rmse:10.28972
[12]	train-rmse:10.57515	eval-rmse:10.18922
[13]	train-rmse:10.53480	eval-rmse:10.14805
[14]	train-rmse:10.49314	eval-rmse:10.10841
[15]	train-rmse:10.40464	eval-rmse:10.02627
[16]	train-rmse:10.37574	eval-rmse:9.99700
[17]	train-rmse:10.35580	eval-rmse:9.97698
[18]	train-rmse:10.27578	eval-rmse:9.93428
[19]	train-rmse:10.19701	eval-rmse:9.85251
[20]	train-rmse:10.16124	eval-rmse:9.82508
[21]	train-rmse:10.12878	eval-rmse:9.79380
[22]	train-rmse:10.09190	eval-rmse:9.76153
[23]	

## XGB Hyperparameter Optimization

In [1]:
from hyperopt import hp, STATUS_OK, fmin, tpe, Trials
import xgboost as xgb

space = {
    'max_depth': hp.choice('max_depth', range(3, 12, 1)),
    'eta': hp.uniform('eta', 0.01, 1),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.3, 1.0),
    'subsample': hp.uniform('subsample', 0.4, 1.0),
    'objective': 'reg:squarederror',
    'eval_metric': 'rmse'  # You can add more metrics relevant to your task
}

def objective(params):
    cv_results = xgb.cv(
        params,
        dtrain,
        num_boost_round=80,
        nfold=5,
        metrics={'rmse'},
        early_stopping_rounds=10,
        seed=42
    )
    loss = min(cv_results['test-rmse-mean'])  # Change rmse to your evaluation metric if different
    return {'loss': loss, 'status': STATUS_OK}

trials = Trials()
best = fmin(
    fn=objective,
    space=space,
    algo=tpe.suggest,
    max_evals = 120,  # You can adjust this number depending on how long you're willing to wait
    trials=trials
)

# best loss after 40 trials: 6.4433288


  0%|          | 0/120 [00:00<?, ?trial/s, best loss=?]

job exception: name 'dtrain' is not defined



  0%|          | 0/120 [00:00<?, ?trial/s, best loss=?]


NameError: name 'dtrain' is not defined

In [13]:
best

{'colsample_bytree': 0.40130062957308954,
 'eta': 0.04912900986421432,
 'max_depth': 2,
 'subsample': 0.7055134369878739}

## LASSO

In [5]:
from sklearn.linear_model import LassoCV

X_train, X_test, y_train, y_test = train_test_split(data, label, test_size=0.1, random_state=42)

del data
del label

reg = LassoCV(cv=5, random_state=0).fit(X_train, y_train)
reg.score(X_train, y_train)

0.4876719268127706

In [5]:
# Saving model
from joblib import dump

dump(reg, r'models\prognostic_model_lasso.joblib')

['models\\prognostic_model_lasso.joblib']

In [5]:
prognostic_lasso = reg.predict(X_members)
print(f"min: {min(prognostic_lasso)}\nmax: {max(prognostic_lasso)}\nmean: {prognostic_lasso.mean()}")

min: -99.65703139608165
max: -3.0157175187344993
mean: -19.66147003221858


In [10]:
# Our models do a bit better than just predicting the mean revenue

pred_naive = y_train.mean()
print("RMSE naive predictions:", np.sqrt(np.mean((pred_naive-y_test)**2)))

RMSE naive predictions: 26.876908670419645
